In [ ]:
import altair as alt
import polars as pl

from altair_utils import color_value_transit_teal
from utils import line_plot


In [ ]:
caltrans_df = pl.read_csv(r"Q:\CMP\LOS Monitoring 2024\vmt.csv")  # from Caltrans
# from database behind covid-congestion prospector dashboard
sfcta_df = (
    pl.read_csv(r"Q:\Model Research\downtown_today\vmt-covid_congestion.csv")
    .with_columns(pl.col("date").str.to_date("%Y-%m"), pl.col("VMT") * 1e6)
    .with_columns(
        year=pl.col("date").dt.year(),
        quarter=pl.col("date").dt.quarter(),
    )
)


In [ ]:
caltrans_chart = line_plot(
    caltrans_df.filter(pl.col("year") > 2017),
    x="year",
    y="VMT",
    title="Daily VMT in San Francisco",
    color=color_value_transit_teal,
    tooltip=["year", "VMT"],
)
caltrans_chart

In [ ]:
sfcta_quarterly_df = (
    sfcta_df.sort("date")
    .group_by("year", "quarter")
    # also convert VMT back to million
    .agg(pl.min("date"), pl.mean("VMT").round_sig_figs(2))
)


In [ ]:
sfcta_chart = (
    alt.Chart(sfcta_quarterly_df)
    .mark_line()
    .encode(
        x=alt.X("date", timeUnit="yearquarter", title="date"),
        y=alt.Y("VMT", title="Daily VMT in San Francisco", axis=alt.Axis(format="s")),
        color=color_value_transit_teal,
        tooltip=["year", "quarter", alt.Tooltip("VMT", format="~s")],
    )
)
sfcta_quarterly_df.write_csv("output/data/vmt.csv")
sfcta_chart.save("output/Links/vmt.png", scale_factor=3)
sfcta_chart.properties(width="container").save("output/Links/vmt.html")
sfcta_chart
